<a href="https://colab.research.google.com/github/O-bit-O/ML-projects/blob/main/Sentiment%20Analysis%20-%20shallow%20vs%20uniLSTM%20vs%20biLSTM/Sentiment_analysis_shallow_vs_uniLSTM_vs_biLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-colab

# Import the library
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

In [ ]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

In [ ]:
# Step 1: Load the dataset
df = pd.read_csv('/content/gdrive/MyDrive/440 dataset/IMDB Dataset.csv')  # Replace with the actual path to dataset

# Step 2: Convert string labels to numerical values
label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])

# Step 3: Use GloVe for word embeddings
embedding_dim = 100
glove_path = '/content/gdrive/MyDrive/440 dataset/glove.6B.100d.txt'  # Replace with the actual path to GloVe file

# Load GloVe embeddings into a dictionary
embedding_index = {}
with open(glove_path, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Create a matrix with GloVe embeddings for our dataset words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['review'])
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# Step 4: Split the data into training and testing sets
X = tokenizer.texts_to_sequences(df['review'])
X = pad_sequences(X)
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 4: Train a basic model

model_basic = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=X.shape[1]),
    Dense(1, activation='sigmoid')
])

model_basic.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_basic.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model_basic.evaluate(X_test, y_test)

# Print the accuracy
print(f'Test Accuracy: {accuracy}')

Epoch 1/20
625/625 [==============================] - 59s 93ms/step - loss: 0.6929 - accuracy: 0.5000 - val_loss: 0.6921 - val_accuracy: 0.5015
Epoch 2/20
625/625 [==============================] - 33s 53ms/step - loss: 0.6921 - accuracy: 0.5054 - val_loss: 0.6920 - val_accuracy: 0.5019
Epoch 3/20
625/625 [==============================] - 23s 37ms/step - loss: 0.6919 - accuracy: 0.5076 - val_loss: 0.6921 - val_accuracy: 0.5020
Epoch 4/20
625/625 [==============================] - 17s 27ms/step - loss: 0.6918 - accuracy: 0.5037 - val_loss: 0.6920 - val_accuracy: 0.5018
Epoch 5/20
625/625 [==============================] - 14s 23ms/step - loss: 0.6917 - accuracy: 0.5019 - val_loss: 0.6922 - val_accuracy: 0.5011
Epoch 6/20
625/625 [==============================] - 13s 21ms/step - loss: 0.6917 - accuracy: 0.5076 - val_loss: 0.6921 - val_accuracy: 0.5019
Epoch 7/20
625/625 [==============================] - 11s 18ms/step - loss: 0.6917 - accuracy: 0.5053 - val_loss: 0.6922 - val_accuracy:

In [ ]:
# Step 5: Train the shallow model
model_shallow = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], input_length=X.shape[1]),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_shallow.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_shallow.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model_shallow.evaluate(X_test, y_test)

# Print the accuracy
print(f'Test Accuracy: {accuracy}')
print(f'Test loss: {loss}')

Epoch 1/20
625/625 [==============================] - 66s 104ms/step - loss: 0.6926 - accuracy: 0.5009 - val_loss: 0.6921 - val_accuracy: 0.5017
Epoch 2/20
625/625 [==============================] - 40s 64ms/step - loss: 0.6920 - accuracy: 0.5036 - val_loss: 0.6919 - val_accuracy: 0.5018
Epoch 3/20
625/625 [==============================] - 27s 42ms/step - loss: 0.6918 - accuracy: 0.5037 - val_loss: 0.6918 - val_accuracy: 0.5020
Epoch 4/20
625/625 [==============================] - 17s 26ms/step - loss: 0.6917 - accuracy: 0.5030 - val_loss: 0.6921 - val_accuracy: 0.5008
Epoch 5/20
625/625 [==============================] - 15s 24ms/step - loss: 0.6916 - accuracy: 0.5055 - val_loss: 0.6920 - val_accuracy: 0.5015
Epoch 6/20
625/625 [==============================] - 12s 19ms/step - loss: 0.6915 - accuracy: 0.5060 - val_loss: 0.6917 - val_accuracy: 0.5021
Epoch 7/20
625/625 [==============================] - 11s 17ms/step - loss: 0.6914 - accuracy: 0.5037 - val_loss: 0.6919 - val_accuracy

In [ ]:
model_lstm_unidirectional = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=X.shape[1]),
    LSTM(100),
    Dense(1, activation='sigmoid')
])

model_lstm_unidirectional.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm_unidirectional.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))
# Evaluate the model on the test set
loss, accuracy = model_lstm_unidirectional.evaluate(X_test, y_test)

# Print the accuracy
print(f'Test Accuracy: {accuracy}')
print(f'Test loss: {loss}')


Epoch 1/20
625/625 [==============================] - 117s 182ms/step - loss: 0.5207 - accuracy: 0.7435 - val_loss: 0.3434 - val_accuracy: 0.8671
Epoch 2/20
625/625 [==============================] - 84s 135ms/step - loss: 0.2524 - accuracy: 0.9001 - val_loss: 0.2498 - val_accuracy: 0.8982
Epoch 3/20
625/625 [==============================] - 75s 120ms/step - loss: 0.1243 - accuracy: 0.9571 - val_loss: 0.2865 - val_accuracy: 0.8930
Epoch 4/20
625/625 [==============================] - 71s 114ms/step - loss: 0.0520 - accuracy: 0.9837 - val_loss: 0.3520 - val_accuracy: 0.8886
Epoch 5/20
625/625 [==============================] - 74s 118ms/step - loss: 0.0194 - accuracy: 0.9949 - val_loss: 0.4618 - val_accuracy: 0.8790
Epoch 6/20
625/625 [==============================] - 74s 118ms/step - loss: 0.0085 - accuracy: 0.9980 - val_loss: 0.5355 - val_accuracy: 0.8789
Epoch 7/20
625/625 [==============================] - 70s 112ms/step - loss: 0.0050 - accuracy: 0.9990 - val_loss: 0.6299 - val_a

In [ ]:
model_lstm_bidirectional = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=X.shape[1]),
    Bidirectional(LSTM(100)),
    Dense(1, activation='sigmoid')
])

model_lstm_bidirectional.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm_bidirectional.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))
# Evaluate the model on the test set
loss, accuracy = model_lstm_bidirectional.evaluate(X_test, y_test)

# Print the accuracy
print(f'Test Accuracy: {accuracy}')
print(f'Test loss: {loss}')

Epoch 1/20
625/625 [==============================] - 160s 250ms/step - loss: 0.4483 - accuracy: 0.7904 - val_loss: 0.2876 - val_accuracy: 0.8842
Epoch 2/20
625/625 [==============================] - 149s 239ms/step - loss: 0.2110 - accuracy: 0.9202 - val_loss: 0.2334 - val_accuracy: 0.9073
Epoch 3/20
625/625 [==============================] - 131s 209ms/step - loss: 0.1009 - accuracy: 0.9668 - val_loss: 0.2710 - val_accuracy: 0.9011
Epoch 4/20
625/625 [==============================] - 135s 217ms/step - loss: 0.0419 - accuracy: 0.9882 - val_loss: 0.3581 - val_accuracy: 0.8884
Epoch 5/20
625/625 [==============================] - 131s 210ms/step - loss: 0.0172 - accuracy: 0.9959 - val_loss: 0.4201 - val_accuracy: 0.8943
Epoch 6/20
625/625 [==============================] - 132s 212ms/step - loss: 0.0076 - accuracy: 0.9982 - val_loss: 0.5863 - val_accuracy: 0.8871
Epoch 7/20
625/625 [==============================] - 122s 194ms/step - loss: 0.0030 - accuracy: 0.9993 - val_loss: 0.6755 -